In [1]:
%pip install torch torchvision
%pip install tqdm
%pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
# Relevant torch imports, TQDM required for visualization
import os
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torch import nn, optim
from tqdm.notebook import tqdm 


print("Package importing successful.")

Package importing successful.


# Loading Dataset
We are using the PlantNet-300K dataset.
TODO: ADD FURTHER DETAILS

In [3]:
# Storing the two transforms.Compose objects for preprocessing
#
# For training: randomly resize+crop and rotate
# For validation: 
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ]),
}

dataset_dir = './images/' 
batch_size = 32
num_workers = 4

# Setting image dataset for training and validation
image_datasets = {x: ImageFolder(os.path.join(dataset_dir, x), data_transforms[x]) for x in ['train', 'val']}

# 
dataloaders = {x: DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=num_workers) for x in ['train', 'val']}

In [4]:
model = torchvision.models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(image_datasets['train'].classes))  # Adjust the final layer based on the number of classes

# if torch.cuda.is_available():
#   device = torch.device("cuda")
# else:
#   device = torch.device("cpu")

print(torch.cuda.is_available())
if torch.cuda.is_available():
    model = model.cuda()
    print("Using CUDA")

C:\Users\asifa\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\asifa\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


True
Using CUDA


In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
num_epochs = 25


In [6]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    running_corrects = 0

    for inputs, labels in tqdm(dataloaders['train'], desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch'):
        if torch.cuda.is_available():
            inputs, labels = inputs.cuda(), labels.cuda()

        optimizer.zero_grad()

        with torch.set_grad_enabled(True):
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    scheduler.step()
    epoch_loss = running_loss / len(image_datasets['train'])
    epoch_acc = running_corrects.double() / len(image_datasets['train'])

    print(f'Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')


Epoch 1/25:   0%|          | 0/7623 [00:00<?, ?batch/s]

Loss: 2.5094 Acc: 0.5138


Epoch 2/25:   0%|          | 0/7623 [00:00<?, ?batch/s]

Loss: 1.6376 Acc: 0.6403


Epoch 3/25:   0%|          | 0/7623 [00:00<?, ?batch/s]

...

Loss: 0.8830 Acc: 0.7765


Epoch 24/25:   0%|          | 0/7623 [00:00<?, ?batch/s]

Loss: 0.8850 Acc: 0.7757


Epoch 25/25:   0%|          | 0/7623 [00:00<?, ?batch/s]

Loss: 0.8813 Acc: 0.7767


In [9]:
model.eval()
val_running_corrects = 0

for inputs, labels in dataloaders['val']:
    if torch.cuda.is_available():
        inputs, labels = inputs.cuda(), labels.cuda()

    with torch.no_grad():
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
    
    val_running_corrects += torch.sum(preds == labels.data)

val_acc = val_running_corrects.double() / len(image_datasets['val'])
print(f'Validation Acc: {val_acc:.4f}')


Validation Acc: 0.7791


In [10]:
torch.save(model.state_dict(), 'saved_models/model.pth')